In [1]:
import pandas as pd
import numpy as np


from statistics import mean, stdev 
from sklearn import preprocessing 
from sklearn.model_selection import StratifiedKFold 
from sklearn import linear_model 
from sklearn import datasets 
from sklearn.model_selection import StratifiedKFold

In [2]:
X_train = pd.read_csv('clean/X_train.csv')
y_train = pd.read_csv('clean/y_train.csv')
X_test = pd.read_csv('clean/X_test.csv')

In [3]:
X_train.head()

,Role,Invested in Real Estate,Extra Hours,Invested in Stock Market,Favourite Sport,Gender,Languages Known,Sport Knowledge (in XP),Average no. of leaves/month,Trades Done,Married(1/0),Level of Education,Pay,ID Proof,Most Productive Hour,Tax Category,Income Category,Employer,Tax Paid,Id
0,1,1,0,1,6,0,1,64.0,2,56,0,2,2416.73,1,8,258,2,0,29597,0
1,0,1,0,1,3,0,3,2.0,0,42,1,0,302.73,1,8,148,2,0,49218,1
2,0,1,1,1,5,1,3,35.0,4,31,0,0,275.73,1,8,73,2,0,14764,2
3,0,1,0,1,4,0,4,70.0,1,45,0,2,1779.73,1,8,89,2,0,46868,3
4,1,0,0,1,5,1,3,25.0,5,31,1,2,274.73,1,8,195,2,0,21859,4


In [4]:
X_test.head()

,Role,Invested in Real Estate,Extra Hours,Invested in Stock Market,Favourite Sport,Gender,Languages Known,Sport Knowledge (in XP),Average no. of leaves/month,Trades Done,Married(1/0),Level of Education,Pay,ID Proof,Most Productive Hour,Tax Category,Income Category,Employer,Tax Paid,Id
0,0,1,0,1,3,1,3,51.0,4,33,0,2,504.73,1,8,136,2,0,47302,5
1,1,1,1,1,5,1,3,93.0,3,26,1,2,720.73,1,8,214,2,0,45732,6
2,0,1,0,1,3,1,1,57.0,4,27,1,0,663.73,1,8,134,2,0,22727,11
3,0,1,0,1,4,0,2,79.0,5,51,0,0,279.73,1,8,514,2,0,40778,12
4,1,1,0,1,2,0,4,41.0,0,56,0,0,344.73,1,8,68,2,0,35447,13


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

param_grid = {'C': [0.1,1, 10], 'gamma': [0.01,0.001]} 

models = [
#     ('decision tree', DecisionTreeClassifier()),
#     ('random forest', RandomForestClassifier(n_estimators = 100, max_depth = 16)),
#     ('knn', KNeighborsClassifier(n_neighbors=15)),
#     ('svc', GridSearchCV(SVC(),param_grid,refit=True,verbose=2)),
    ('xgboost', XGBClassifier(max_depth = 16))
#     ('log reg', LogisticRegression())
]

In [16]:
def evaluate(model, X, y, test):
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1) 
    scores = []
    
    x_scaled = pd.DataFrame(preprocessing.normalize(X))

    for train_index, test_index in skf.split(X, y): 
#         print(train_index, test_index)
        x_train_fold, x_test_fold = x_scaled.iloc[train_index], x_scaled.iloc[test_index] 
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index] 
        
        model.fit(x_train_fold, y_train_fold) 
        preds = model.predict(x_test_fold)
#         print(set(preds))
        scores.append(model.score(x_test_fold, y_test_fold)) 
    
    test_scaled = preprocessing.normalize(test)
    
    score = mean(scores)
    preds = model.predict(test_scaled)

    return score, preds

In [ ]:
scores = []
for name, model in models:
    get = evaluate(model, X_train, y_train, X_test)
    scores.append([get[0], name, get[1]])
    print(name, get[0])
    print(set(get[1]))

/home/nachiket/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nachiket/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[18:41:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nachiket/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nachiket/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[18:42:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
out = scores[0][2]
print(out)

In [ ]:
pd.DataFrame({
    'Id' : X_test['Id'],
    'Occupation': out
}).to_csv('new_out/xgb.csv', index = False)